In [35]:
import os
import pickle
from sparta.Auxil.PeriodicityDetector import PeriodicityDetector
from sparta.UNICOR.Spectrum import Spectrum
from sparta.UNICOR.Template import Template
from sparta.Auxil.TimeSeries import TimeSeries
from sparta.Observations import Observations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import imageio

In [124]:
# DATA PARAMETERS

survey = "HERMES" # survey name
sample_rate = 1 # sample rate of the data
min_wv = 5800 # minimum wavelength of the data
max_wv = 6000 # maximum wavelength of the data
spec_dir = r"C:\Users\kentb\Desktop\PDM\data\Hermes\spectra\delCep_Hermes_spectra" # directory of the spectrum
obs_data = Observations(survey=survey, sample_rate=sample_rate, min_wv=min_wv, max_wv=max_wv,
                        target_visits_lib=spec_dir)
period_truth = 5.366267

let's goooo
Using target_visits_lib: C:\Users\kentb\Desktop\PDM\data\Hermes\spectra\delCep_Hermes_spectra
Selected directory: C:\Users\kentb\Desktop\PDM\data\Hermes\spectra\delCep_Hermes_spectra


In [125]:
# Preprocess the spectra
for i in obs_data.time_series.vals:
    i = i.InterpolateSpectrum()
    i = i.FilterSpectrum(lowcut=3, highcut=0.15, order = 1)
    i = i.ApplyCosineBell(alpha=0.3)


In [126]:
#fold the times and get the new order of the indices

times_folded = [t%period_truth for t in obs_data.time_series.times]
print(obs_data.time_series.times)
print(times_folded)
print(obs_data.time_series.vals)

indices = np.argsort(times_folded).tolist()
obs_data.time_series.times = [obs_data.time_series.times[i] for i in indices]
obs_data.time_series.vals = [obs_data.time_series.vals[i] for i in indices]
print([t%period_truth for t in obs_data.time_series.times])
print(obs_data.time_series.vals)
print(len(obs_data.time_series.vals), len(obs_data.time_series.times),len(times_folded))

[0.4696271000429988, 0.47091960022225976, 0.472220900002867, 1.5106675000861287, 1.5119612999260426, 2.4940936001949012, 2.495383800007403, 2.496674600057304, 3.4972692001610994, 3.4985635997727513, 3.4998534000478685, 80.40278209978715, 80.40372390020639, 80.40466499980539, 81.34205149998888, 81.34299910021946, 81.34394010016695, 82.35071639996022, 82.35168959992006, 82.35266030021012, 83.36490250006318, 83.36585310008377, 83.36680560000241, 84.38721860013902, 84.38886319985613, 84.39084930019453, 85.42809440009296, 85.42973939981312, 85.43190089985728, 459.32676269998774, 459.379255999811, 459.4349763998762, 460.3256545001641, 460.39792690007016, 460.44895630003884, 461.3287610998377, 461.4223584998399, 461.4560197000392, 462.3421204001643, 462.38004139997065, 462.4382977997884, 463.3251435002312, 463.3603921998292, 463.40713030006737, 464.32164219999686, 464.3601490999572, 464.39432819979265, 466.3334127999842, 466.41657700017095, 467.32335660001263, 467.38501340011135, 467.42033419

In [132]:
# Initialize plot
fig, ax = plt.subplots()
print(len(obs_data.time_series.vals[0].wv[0]), len(obs_data.time_series.vals[0].sp[0]),len(obs_data.time_series.vals))
wavelengths = obs_data.time_series.vals[0].wv[0]
line, = ax.plot(wavelengths, obs_data.time_series.vals[0].sp[0], lw=1, color='k')

# Animation function
def update(frame):
    line.set_xdata(obs_data.time_series.vals[frame].wv[0])
    line.set_ydata(obs_data.time_series.vals[frame].sp[0])
    max_y = np.max(obs_data.time_series.vals[frame].sp[0])
    min_y = np.min(obs_data.time_series.vals[frame].sp[0])
    min_x = 5887
    max_x = 5897
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)
    ax.set_xlabel(r'Wavelength $\AA$')
    ax.set_ylabel('Intensity')
    ax.set_title(r'$Na$, $\delta$ Cep',loc='left',fontsize=8)
    ax.set_title('Time: ' + str([t%period_truth for t in obs_data.time_series.times][frame]) + ' days',loc='right',fontsize=6)
    return line,

# Set animation properties
ani = FuncAnimation(fig, update, frames=len(obs_data.time_series.vals),
                    interval=50, blit=True)

# Enable interactive mode for the notebook
%matplotlib notebook

# Display the animation
#HTML(ani.to_jshtml())

ani.save('delCep_NaII.gif', writer='pillow', fps=8)

<IPython.core.display.Javascript object>

13035 13035 288


Traceback (most recent call last):
  File "c:\Users\kentb\miniconda3\envs\pdm\lib\site-packages\matplotlib\cbook\__init__.py", line 216, in process
    func(*args, **kwargs)
  File "c:\Users\kentb\miniconda3\envs\pdm\lib\site-packages\matplotlib\animation.py", line 1465, in _stop
    self.event_source.remove_callback(self._loop_delay)
AttributeError: 'NoneType' object has no attribute 'remove_callback'


In [131]:
plt.figure()
plt.plot(obs_data.time_series.vals[0].wv[0], obs_data.time_series.vals[0].sp[0], lw=1, color='k')
plt.show()

<IPython.core.display.Javascript object>